In [1]:
import numpy as np
import matplotlib.pyplot as plt
from math import pi
from nFNC_functions import *
from Entrada_de_dados import *
from ELUi_functions import *

ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELUi ok!
False 0.16666666666666666 0.03151859892064612


In [2]:
fck, gamma_c, sigma_cd, gamma_conc, f_yk, gamma_s, E_s, f_yd, epsilon_yd, gamma_aco, c, b, h, d, nc, nb, phi, y_s, N_d, M_d, epsilon_c2, epsilon_cu, x_lim, n, tol_J, tol_k, tol_f, i, it_max, y_t, y_b, epsilon_0, k, epsilon_0_it, k_it, epsilon_t, epsilon_b = Entrada_de_dados()
m = 5
l_e = 20*h

In [3]:
Rompeu, e, f = Verificacao_DF(fck, gamma_c, sigma_cd, gamma_conc, f_yk, gamma_s, E_s, f_yd, epsilon_yd, gamma_aco, c, b, h, d, nc, nb, phi, y_s, N_d, M_d, epsilon_c2, epsilon_cu, x_lim, n, tol_J, tol_k, tol_f, i, it_max, y_t, y_b, epsilon_0, k, epsilon_0_it, k_it, epsilon_t, epsilon_b, m, l_e)
print(Rompeu, e, f)

ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELU ok!
ELUi ok!
False 0.16666666666666666 0.03151859892064612


# Verificação da estabilidade de um pilar

In [4]:
# Arbitra-se uma flecha tentativa inicial f = 0
f = 0
i=0
N = F/(b*h)
M = N*(e+f)
Rompeu, epsilon_0, k, epsilon_0_it, k_it, f_ad = Verificacao(fck, gamma_c, sigma_cd, gamma_conc, f_yk, gamma_s, E_s, f_yd, epsilon_yd, gamma_aco, c, b, h, d, nc, nb, phi, y_s, N, M, epsilon_c2, epsilon_cu, x_lim, n, tol_J, tol_k, tol_f, i, it_max, y_t, y_b, epsilon_0, k, epsilon_0_it, k_it, epsilon_t, epsilon_b)
if Rompeu:
    print('Failure due to lack of resistant capacity')
    return False, e, f
while (True):
    N = N*np.ones(m)
    y = M = _r = np.zeros(m)

    for i in range(m):
        if (i == 0):
            y[i] = 0
            _r[i] = k/1000
            M[i] = N[i]*(e+f-y[i])
        elif (i == 1):
            y[i] = _r*dL**2/2
            M[i] = N[i]*(e+f-y[i])
            Rompeu, epsilon_0, k, epsilon_0_it, k_it, f_ad = Verificacao(fck, gamma_c, sigma_cd, gamma_conc, f_yk, gamma_s, E_s, f_yd, epsilon_yd, gamma_aco, c, b, h, d, nc, nb, phi, y_s, N[i], M[i], epsilon_c2, epsilon_cu, x_lim, n, tol_J, tol_k, tol_f, i, it_max, y_t, y_b, epsilon_0, k, epsilon_0_it, k_it, epsilon_t, epsilon_b)
            _r[i] = k/1000
        else:
            y[i] = _r[i-1]*dL**2+2*y[i-1]-y[i-2]
            M[i] = N[i]*(e+f-y[i])
            Rompeu, epsilon_0, k, epsilon_0_it, k_it, f_ad = Verificacao(fck, gamma_c, sigma_cd, gamma_conc, f_yk, gamma_s, E_s, f_yd, epsilon_yd, gamma_aco, c, b, h, d, nc, nb, phi, y_s, N[i], M[i], epsilon_c2, epsilon_cu, x_lim, n, tol_J, tol_k, tol_f, i, it_max, y_t, y_b, epsilon_0, k, epsilon_0_it, k_it, epsilon_t, epsilon_b)
            _r[i] = k/1000
        if Rompeu:
            print('Failure due to lack of resistant capacity')
            return True, e, f
    
    if abs(y[m] - f) <= h*tol_DF:
        print('ELUi ok!')
        return False, e, f
    else:
        f = y[m]
            
        


NameError: name 'F' is not defined